In [1]:
'''
本部分是基于依存句法分析提取文本中的(实体,关系,实体)三元组的代码，主要借助的是百度的DDParser工具
'''
import os, re
from ddparser import DDParser


In [2]:
# 基于百度DDParser依存句法分析的SVO三元组抽取
class SVOParser:
    def __init__(self):
        self.parser = DDParser(use_pos=True) # 依存句法分析器
        print('loaded model')

    '''
    文章分句处理, 切分长句，冒号，分号，感叹号等做切分标识
    '''

    def split_sents(self, content):
        return [sentence for sentence in re.split(r'[？?！!。；;：:\n\r]', content) if sentence]

    '''
    句法分析---为句子中的每个词语维护一个保存句法依存儿子节点的字典    
    '''
    def build_parse_child_dict(self, words, postags, rel_id, relation):
        child_dict_list = [] # 保存句法依存儿子节点的字典
        format_parse_list = [] # 保存格式化后的依存关系
        for index in range(len(words)):
            child_dict = dict() # 空字典child_dict
            for arc_index in range(len(rel_id)): # 使用arc_index变量迭代rel_id列表的索引
                if rel_id[arc_index] == index+1:   # 检查arc_index处的rel_id值是否等于index+1，如果是，则表示依赖关系对应于索引处的当前单词。然后，代码相应地更新child_dict
                    if rel_id[arc_index] in child_dict: # 如果当前的rel_id值已经作为关键字存在于child_dict中，则arc_index会附加到相应的索引列表中
                        child_dict[relation[arc_index]].append(arc_index)
                    else: # 否则，将创建一个新键，并将其值初始化为包含arc_index的空列表
                        child_dict[relation[arc_index]] = []
                        child_dict[relation[arc_index]].append(arc_index)
            child_dict_list.append(child_dict) # 在处理了当前单词的所有依赖关系后，child_dict被附加到child_dict_list
        heads = ['Root' if id == 0 else words[id - 1] for id in rel_id]  # 将每个依赖关系id与其对应的单词匹配，若id为0则为根节点
        for i in range(len(words)):
            a = [relation[i], words[i], i, postags[i], heads[i], rel_id[i]-1, postags[rel_id[i]-1]] # 为每个单词创建解析信息的格式化表示
            format_parse_list.append(a)
        # child_dict_list包含每个单词的依赖关系，format_parse_list包含每个单词的格式化解析信息
        return child_dict_list, format_parse_list

    '''
    parser主函数
    '''
    def parser_main(self, sentence):
        res = self.parser.parse(sentence, )[0] # 调用DDParser的parse方法，对句子进行依存句法分析
        # 分别将解析的结果提取出来
        words = res["word"]
        postags = res["postag"]
        rel_id = res["head"]
        relation = res["deprel"]
        # 根据提取的信息构建解析树结构，并返回两个值：child_dict_list 和 format_parse_list
        child_dict_list, format_parse_list = self.build_parse_child_dict(words, postags, rel_id, relation) # 调用build_parse_child_dict方法，为句子中的每个词语维护一个保存句法依存儿子节点的字典
        return words, postags, child_dict_list, format_parse_list # 该方法返回解析后的词语、词性标注、以子字典列表形式表示的解析树（child_dict_list）以及格式化的解析列表（format_parse_list）

    """
    将所有的ATT进行合并
    """
    def merge_ATT(self, words, postags, format_parse_list):
        words_ = words # 浅拷贝
        retain_nodes = set()
        ATTs = []
        ATT = []
        format_parse_list_ = []
        for parse in format_parse_list: # 遍历format_parse_list集合，每个parse都是一个包含特定依存关系信息的列表
            dep = parse[0]
            if dep in ['ATT', 'ADV']: # 如果依存关系是ATT或ADV，则将其添加到ATT列表中
                ATT += [parse[2], parse[5]]
            else: # 如果依存类型不是 'ATT' 或 'ADV'，代码会检查 ATT 列表是否为空
                if ATT: # 如果ATT列表不为空，则将其合并为一个字符串，并将其添加到ATTs列表中
                    body = ''.join([words[i] for i in sorted(set(ATT))])
                    ATTs.append(body) # 将 ATT 列表中索引对应的词连接起来形成一个单词（body），并将其添加到 ATTs 列表中
                    retain_nodes.add(sorted(set(ATT))[-1]) # 将 ATT 列表中最后一个索引添加到retain_nodes集合中
                    words_[sorted(set(ATT))[-1]] = body # 将 ATT 列表中最后一个索引对应的词替换为body
                else:
                    retain_nodes.add(parse[2]) # 如果 ATT 列表为空，意味着序列中没有 'ATT' 或 'ADV' 依存关系，此时只需将当前 parse 的索引添加到 retain_nodes 集合中即可
                ATT = []
        for indx, parse in enumerate(format_parse_list):
            if indx in retain_nodes: # 检查当前索引（indx）是否存在于 retain_nodes 集合中
                parse_ = [parse[0], words_[indx], indx, postags[indx], words_[parse[5]], parse[5], postags[parse[5]]] # 如果存在，则将该索引对应的词替换为words_[indx]
                format_parse_list_.append(parse_) # 将该parse_添加到format_parse_list_中
        return words_, postags, format_parse_list_, retain_nodes

    """
    根据词性标注（postags）、依存解析（child_dict_list）和其他语言信息从给定的句子中提取语义三元组（主语-动词-宾语关系）
    该方法旨在通过利用词性标注和依存解析信息从句子中提取主谓宾和主谓补关系，提取的三元组提供了关于句子的语义结构和关系的见解
    """
    def extract(self, words, postags, child_dict_list, arcs, retain_nodes):
        svos = [] # 用于保存抽取出的三元组
        for index in range(len(postags)): # 遍历句子中的每个词
            if index not in retain_nodes: # 如果该词不在retain_nodes集合中，则跳过该词 -- 只考虑对句子中的特定节点进行提取
                continue
            # 检查给定索引处的当前词语是否具有非空的词性标注（postags[index]）。如果不为空，则根据句法结构提取语义三元组
            if postags[index]:
                child_dict = child_dict_list[index] # 获取当前词语的依存解析信息
                # 查找主谓宾（SVO）三元组，通过检查是否存在主语（SBV）和宾语（VOB）作为子节点来确定
                if 'SBV' in child_dict and 'VOB' in child_dict:
                    r = words[index] # 提取谓语(动词)
                    e1 = words[child_dict['SBV'][0]] # 提取主语
                    e2 = words[child_dict['VOB'][0]] # 提取宾语
                    if e1.replace(' ', '') and e2.replace(' ', ''): 
                        svos.append([e1, r, e2]) # 提取的主语（e1）、动词（r）和宾语（e2）以语义三元组[e1, r, e2]的形式添加到svos列表中，分别表示主语、动词和宾语

                # 通过查找主语（SBV）和补语（CMP）关系来检查主谓补（SVC）三元组是否存在
                if 'SBV' in child_dict and 'CMP' in child_dict:
                    e1 = words[child_dict['SBV'][0]] # 提取主语
                    cmp_index = child_dict['CMP'][0] # 提取补语
                    r = words[index] + words[cmp_index] # 将谓词和补语连接起来形成动词(谓词)
                    if 'POB' in child_dict_list[cmp_index]: # 检查补语的子节点是否存在介宾关系（POB）
                        e2 = words[child_dict_list[cmp_index]['POB'][0]] # 如果存在介宾关系，则提取介宾关系对应的宾语
                        if e1.replace(' ', '') and e2.replace(' ', ''):
                            svos.append([e1, r, e2]) # 提取的主语（e1）、谓词（r）和宾语（e2）以语义三元组[e1, r, e2]的形式添加到svos列表中

        return svos

    '''
    三元组抽取主函数，旨在根据词性标注和依存解析信息从句子中提取主谓宾、主谓补和定语后置的动宾关系
    '''
    def ruler2(self, words, postags, child_dict_list, arcs):
        svos = [] # 用于保存抽取出的三元组
        for index in range(len(postags)): # 遍历句子中的每个词
            tmp = 1
            if tmp == 1: # 这个条件总是为真，因此这个条件是多余的...
                if postags[index]: # 如果语义角色标注（postags[index]）不为空，则使用语义角色标注的结果进行三元组抽取
                    child_dict = child_dict_list[index]
                    if 'SBV' in child_dict and 'VOB' in child_dict: # 查找主谓宾（SVO）三元组，通过检查是否存在主语（SBV）和宾语（VOB）作为子节点来确定
                        r = words[index] # 提取谓语(动词)
                        e1 = self.complete_e(words, postags, child_dict_list, child_dict['SBV'][0]) # 提取主语
                        e2 = self.complete_e(words, postags, child_dict_list, child_dict['VOB'][0]) # 提取宾语
                        if e1.replace(' ', '') and e2.replace(' ', ''): 
                            svos.append([e1, r, e2]) # 提取的主语（e1）、动词（r）和宾语（e2）以语义三元组[e1, r, e2]的形式添加到svos列表中，分别表示主语、动词和宾语

                    relation = arcs[index][0]
                    head = arcs[index][2]
                    if relation == 'ATT': # 检查当前词语的依存关系（relation）是否为“ATT”（定语后置），并且检查该词语的头部词语索引（head）
                        if 'VOB' in child_dict: # 进一步检查是否存在宾语（VOB）作为子节点
                            # 若存在，则使用complete_e方法来获取完整的实体词语，并构建谓词（r）
                            e1 = self.complete_e(words, postags, child_dict_list, head - 1) 
                            r = words[index]
                            e2 = self.complete_e(words, postags, child_dict_list, child_dict['VOB'][0])
                            # 构建一个临时字符串（temp_string），由谓词和宾语拼接而成。然后检查主语是否以该临时字符串开头，如果是，则从主语中移除该临时字符串的部分
                            temp_string = r + e2
                            if temp_string == e1[:len(temp_string)]:
                                e1 = e1[len(temp_string):]
                            # 检查临时字符串是否不在主语中，如果是，则将主语、谓词和宾语以语义三元组的形式添加到svos列表中
                            if temp_string not in e1:
                                if e1.replace(' ', '') and e2.replace(' ', ''):
                                    svos.append([e1, r, e2])


                    if 'SBV' in child_dict and 'CMP' in child_dict: # 通过查找主语（SBV）和补语（CMP）关系来检查主谓补（SVC）三元组是否存在
                        e1 = self.complete_e(words, postags, child_dict_list, child_dict['SBV'][0]) # 提取主语
                        cmp_index = child_dict['CMP'][0] # 提取补语
                        r = words[index] + words[cmp_index] # 将谓词和补语连接起来形成动词(谓词)
                        if 'POB' in child_dict_list[cmp_index]: # 检查补语的子节点是否存在介宾关系（POB）
                            e2 = self.complete_e(words, postags, child_dict_list, child_dict_list[cmp_index]['POB'][0]) # 如果存在介宾关系，则提取介宾关系对应的宾语
                            if e1.replace(' ', '') and e2.replace(' ', ''):
                                svos.append([e1, r, e2]) # 提取的主语（e1）、谓词（r）和宾语（e2）以语义三元组[e1, r, e2]的形式添加到svos列表中
        return svos

    '''
    对找出的主语或者宾语进行扩展
    通过递归调用自身，根据依存解析和词性标注信息，构建一个完整的实体词语，其中包括定语、动宾关系和主谓关系等相关部分
    '''
    def complete_e(self, words, postags, child_dict_list, word_index):
        child_dict = child_dict_list[word_index] # 获取词语索引对应的子节点字典
        prefix = '' # 存储前缀
        # 如果 child_dict 中存在 'ATT'（定语）关系，则通过循环遍历所有定语关系的子节点，使用递归调用 complete_e 方法获取每个子节点的完整实体词语，并将其添加到 prefix 中
        if 'ATT' in child_dict:
            for i in range(len(child_dict['ATT'])):
                prefix += self.complete_e(words, postags, child_dict_list, child_dict['ATT'][i])
        postfix = '' # 存储后缀
        # 如果 postags[word_index] 为 'v'（动词）
        if postags[word_index] == 'v':
            if 'VOB' in child_dict: # 如果 child_dict 中存在 'VOB'（动宾关系），则通过递归调用 complete_e 方法获取宾语的完整实体词语，并将其添加到 postfix 中
                postfix += self.complete_e(words, postags, child_dict_list, child_dict['VOB'][0])
            if 'SBV' in child_dict: # 如果 child_dict 中存在 'SBV'（主谓关系），则通过递归调用 complete_e 方法获取主语的完整实体词语，并将其与 prefix 进行拼接
                prefix = self.complete_e(words, postags, child_dict_list, child_dict['SBV'][0]) + prefix
        # 返回拼接后的字符串，由 prefix、words[word_index] 和 postfix 组成，表示完整的实体词语
        return prefix + words[word_index] + postfix

    '''
    程序主控函数，调用类方法依次实现分句、依存分析和三元组提取
    '''
    def triples_main(self, content):
        sentences = self.split_sents(content)
        svos = []
        for sentence in sentences:
            # print(sentence)
            words, postags, child_dict_list, arcs = self.parser_main(sentence)
            svo = self.ruler2(words, postags, child_dict_list, arcs)
            svos += svo

        return svos

In [3]:
# 新闻标题文本三元组抽取
def title_triples__ext():
    extractor_1 = SVOParser()
    target_file = open('./data/title_triples_data.txt', 'w', encoding='utf-8')
    with open('./data/title_data.txt','r',encoding='utf-8') as f:
        for line in f:
            svos = extractor_1.triples_main(line)
            for svo in svos:
                print(svo)
                target_file.write(str(svo) + '\n')
    target_file.close()
    print('新闻标题文本三元组提取完成')

In [4]:
# 正文文本三元组抽取
def text_triples__ext():
    extractor_2 = SVOParser()
    target_file = open('./data/text_triples_data.txt', 'w', encoding='utf-8')
    with open('./data/text_data.txt','r',encoding='utf-8') as f:
        for line in f:
            svos = extractor_2.triples_main(line)
            for svo in svos:
                print(svo)
                target_file.write(str(svo) + '\n')
    target_file.close()
    print('正文文本三元组提取完成')

In [5]:
# 标题和正文的三元组提取完毕后，将两个文件合并
# 将title_data和text_data合并
def merge_triples_data():

    with open('./data/text_triples_data.txt', 'r',encoding='utf-8') as file1:
        text1 = file1.read()

    with open('./data/title_triples_data.txt', 'r',encoding='utf-8') as file2:
        text2 = file2.read()

    merged_text = text1 + text2

    with open('./data/triples_data.txt', 'w',encoding='utf-8') as merged_file:
        merged_file.write(merged_text)
    
    print('合并完成')


In [6]:
# 分别执行标题文本和正文文本三元组抽取，抽取完毕后将两个文件合并
if __name__ == '__main__':
    title_triples__ext()
    text_triples__ext()
    merge_triples_data()

loaded model


d:\Download_software\My_miniconda\envs\pytorch_py3.7\lib\site-packages\paddle\fluid\dygraph\math_op_patch.py:277: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.int64, but right dtype is paddle.int32, the right dtype will convert to paddle.int64
  .format(lhs_dtype, rhs_dtype, lhs_dtype))
d:\Download_software\My_miniconda\envs\pytorch_py3.7\lib\site-packages\paddle\fluid\framework.py:4006: DeprecationWarning: Op `cumsum` is executed through `append_op` under the dynamic mode, the corresponding API implementation needs to be upgraded to using `_C_ops` method.
  DeprecationWarning,
d:\Download_software\My_miniconda\envs\pytorch_py3.7\lib\site-packages\paddle\fluid\dygraph\math_op_patch.py:277: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.int64, but right dtype is paddle.bool, the right dtype will convert to paddle.int64
  .format(lhs_dtype, rhs_dtype, lhs_dtype))
d:\Download_software\My_miniconda\envs

['377人', '失踪', '94人']
['暴雨灾害', '致', '150万余人受灾']
['暴雨', '致', '70人死亡']
['16省份', '遇', '洪涝灾害']
['洪涝灾害', '造成', '24省份受灾']
['洪涝灾害', '导致', '5793.1万人受灾']
['麦德姆', '致', '受灾人口1.83万']
['宜昌夷陵区', '遭', '特大洪灾']
['4省份', '发生', '洪涝灾害']
['洪灾', '致', '85人死亡']
['已', '致', '85人死亡']
['洪灾致85人死亡习近平李克强', '做出', '指示']
['洪涝灾情', '持续', '3人死亡']
['已', '致', '内蒙辽宁']
['东北华南', '遭受', '54人']
['东北局部地区', '遭受', '洪涝灾害']
['东北局部地区', '遭受', '洪涝灾害']
['东北局部地区遭受洪涝灾害', '造成', '1人死亡']
['洪涝灾害', '致', '85人死亡']
['已', '致', '85人死亡']
['强降雨', '致', '洪涝灾害']
['两部委', '补助', '资金']
['两部门', '启动', '四级应急响应']
['两部门', '启动', 'Ⅳ级应急响应']
['两部门', '启动', '四级救灾应急响应']
['两部门', '启动', 'Ⅳ级应急响应']
['两部门', '启动', 'Ⅳ级救灾应急响应']
['两部门', '启动', 'Ⅳ级救灾应急响应']
['两部门', '启动', '国家四级应急响应']
['两部门', '启动', '四级救灾应急响应']
['两部门', '启动', '国家四级救灾应急响应']
['中国', '遭', '暴雨']
['中国遭暴雨', '造成', '6人失踪']
['28个省份', '遭受', '洪涝灾害']
['受灾人口', '达', '1.4亿人']
['28个省份', '遭受', '洪灾']
['28个省份', '遭受', '洪灾']
['1.37亿人', '遭受', '558人失踪']
['28省份', '遭', '928人']
['28省区市', '遭受', '742人洪涝灾害死亡']
['降水', '造成', '洪涝滑坡灾害']
['中央财政', '拨', '3亿元